In [ ]:
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TEST").getOrCreate()
spark

In [ ]:
!mkdir data
!wget -O data/user.parquet https://eurocash-pyspark.s3.eu-central-1.amazonaws.com/userdata1.parquet

df = spark.read.parquet("data/user.parquet")
df.printSchema()

--2023-06-19 10:04:23--  https://eurocash-pyspark.s3.eu-central-1.amazonaws.com/userdata1.parquet
Resolving eurocash-pyspark.s3.eu-central-1.amazonaws.com (eurocash-pyspark.s3.eu-central-1.amazonaws.com)... 52.219.169.130
Connecting to eurocash-pyspark.s3.eu-central-1.amazonaws.com (eurocash-pyspark.s3.eu-central-1.amazonaws.com)|52.219.169.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113629 (111K) [binary/octet-stream]
Saving to: ‘data/user.parquet’

data/user.parquet   100%[===================>] 110.97K   374KB/s    in 0.3s    

2023-06-19 10:04:24 (374 KB/s) - ‘data/user.parquet’ saved [113629/113629]

root
 |-- registration_dttm: timestamp (nullable = true)
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- cc: string (nullable = true)
 |-- country: string (nullab

In [ ]:
df.select("registration_dttm","first_name","last_name").show(n=5)

+-------------------+----------+---------+
|  registration_dttm|first_name|last_name|
+-------------------+----------+---------+
|2016-02-03 07:55:29|    Amanda|   Jordan|
|2016-02-03 17:04:03|    Albert|  Freeman|
|2016-02-03 01:09:31|    Evelyn|   Morgan|
|2016-02-03 00:36:21|    Denise|    Riley|
|2016-02-03 05:05:31|    Carlos|    Burns|
+-------------------+----------+---------+
only showing top 5 rows



In [ ]:
df.select(df["registration_dttm"], df["first_name"], df["last_name"]).show(n=5)

+-------------------+----------+---------+
|  registration_dttm|first_name|last_name|
+-------------------+----------+---------+
|2016-02-03 07:55:29|    Amanda|   Jordan|
|2016-02-03 17:04:03|    Albert|  Freeman|
|2016-02-03 01:09:31|    Evelyn|   Morgan|
|2016-02-03 00:36:21|    Denise|    Riley|
|2016-02-03 05:05:31|    Carlos|    Burns|
+-------------------+----------+---------+
only showing top 5 rows



In [ ]:
df.select(df.registration_dttm, df.first_name, df.last_name).show(n=5)

+-------------------+----------+---------+
|  registration_dttm|first_name|last_name|
+-------------------+----------+---------+
|2016-02-03 07:55:29|    Amanda|   Jordan|
|2016-02-03 17:04:03|    Albert|  Freeman|
|2016-02-03 01:09:31|    Evelyn|   Morgan|
|2016-02-03 00:36:21|    Denise|    Riley|
|2016-02-03 05:05:31|    Carlos|    Burns|
+-------------------+----------+---------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col

df.select( col("registration_dttm"), col("first_name"), col("last_name") ).show(n=3)

+-------------------+----------+---------+
|  registration_dttm|first_name|last_name|
+-------------------+----------+---------+
|2016-02-03 07:55:29|    Amanda|   Jordan|
|2016-02-03 17:04:03|    Albert|  Freeman|
|2016-02-03 01:09:31|    Evelyn|   Morgan|
+-------------------+----------+---------+
only showing top 3 rows



In [ ]:
df.select( df.colRegex("`.*name*`") ).show(n=5)

+----------+---------+
|first_name|last_name|
+----------+---------+
|    Amanda|   Jordan|
|    Albert|  Freeman|
|    Evelyn|   Morgan|
|    Denise|    Riley|
|    Carlos|    Burns|
+----------+---------+
only showing top 5 rows



In [ ]:
df.select("*").show(n=3)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+-------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|  country|birthdate|   salary|              title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+-------------------+--------+
|2016-02-03 07:55:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|   Internal Auditor|   1E+02|
|2016-02-03 17:04:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|      Accountant IV|        |
|2016-02-03 01:09:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|   Russia| 2/1/1960|144972.51|Structural Engineer|        |
+---

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

data = [
    ( ("Jan","Kowalski") , 44, "M"  ),
    ( ("Marian","Nowak") , 33, "K"  ),
]

schema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)



In [ ]:
df.show(n=5)

+---------------+---+------+
|           name|age|gender|
+---------------+---+------+
|{Jan, Kowalski}| 44|     M|
|{Marian, Nowak}| 33|     K|
+---------------+---+------+



In [ ]:
#df.select("name").show()
df.select("name.firstname", "name.lastname").show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|      Jan|Kowalski|
|   Marian|   Nowak|
+---------+--------+



In [ ]:
df.select("name.*", "age").show()

+---------+--------+---+
|firstname|lastname|age|
+---------+--------+---+
|      Jan|Kowalski| 44|
|   Marian|   Nowak| 33|
+---------+--------+---+



In [ ]:
df.schema.jsonValue()

{'type': 'struct',
 'fields': [{'name': 'name',
   'type': {'type': 'struct',
    'fields': [{'name': 'firstname',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'lastname',
      'type': 'string',
      'nullable': True,
      'metadata': {}}]},
   'nullable': True,
   'metadata': {}},
  {'name': 'age', 'type': 'integer', 'nullable': True, 'metadata': {}},
  {'name': 'gender', 'type': 'string', 'nullable': True, 'metadata': {}}]}

In [ ]:
df1 = df.select( col("name.firstname") , col("name.lastname").alias("LastName"),\
          df.age.cast("float"), (df.age>=40).alias("over_40") )
df1.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- age: float (nullable = true)
 |-- over_40: boolean (nullable = true)



In [ ]:
data = [
    (2, "Jan","Kowalski" , "M"  ),
    (1, "Maria","Nowak" , "K"  ),
     (5, "Zenon","Martyniuk" , None  ),
     (10, "Grzegorz","Braun" , "M"  ),
]

columns = ["id", "fname", "lname", "gender"]

df = spark.createDataFrame(data, columns)
df.show()

+---+--------+---------+------+
| id|   fname|    lname|gender|
+---+--------+---------+------+
|  2|     Jan| Kowalski|     M|
|  1|   Maria|    Nowak|     K|
|  5|   Zenon|Martyniuk|  null|
| 10|Grzegorz|    Braun|     M|
+---+--------+---------+------+



In [ ]:
# filtrowanie
df.filter( df.id>2 ).show()

+---+--------+---------+------+
| id|   fname|    lname|gender|
+---+--------+---------+------+
|  5|   Zenon|Martyniuk|  null|
| 10|Grzegorz|    Braun|     M|
+---+--------+---------+------+



In [ ]:
%%timeit -r 5 -n 200
df.filter( (df.id>=3) & (df.id<=9) )

2.66 ms ± 158 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [ ]:
%%timeit -r 5 -n 200
df.filter( df.id.between(3, 9) )

2.12 ms ± 90.7 µs per loop (mean ± std. dev. of 5 runs, 200 loops each)


In [ ]:
df.filter( df.lname.contains("wa") ).show()

+---+-----+--------+------+
| id|fname|   lname|gender|
+---+-----+--------+------+
|  2|  Jan|Kowalski|     M|
|  1|Maria|   Nowak|     K|
+---+-----+--------+------+



In [ ]:
df.filter( df.fname.endswith("a") ).show()

+---+-----+-----+------+
| id|fname|lname|gender|
+---+-----+-----+------+
|  1|Maria|Nowak|     K|
+---+-----+-----+------+



In [ ]:
df.filter( df.gender.isNull() ).show()

+---+-----+---------+------+
| id|fname|    lname|gender|
+---+-----+---------+------+
|  5|Zenon|Martyniuk|  null|
+---+-----+---------+------+



In [ ]:
df.filter( df.gender.isNotNull() ).show()

+---+--------+--------+------+
| id|   fname|   lname|gender|
+---+--------+--------+------+
|  2|     Jan|Kowalski|     M|
|  1|   Maria|   Nowak|     K|
| 10|Grzegorz|   Braun|     M|
+---+--------+--------+------+



In [ ]:
from pyspark.sql.functions import upper, when
df.select( upper(df.fname.substr(1,2)).alias("FN"), df.lname.substr(1,2).alias("LN") ).show()

+---+---+
| FN| LN|
+---+---+
| JA| Ko|
| MA| No|
| ZE| Ma|
| GR| Br|
+---+---+



In [ ]:
df.select( df.fname, df.lname, \
          when(df.gender=="K", "Kobieta").when(df.gender=="M","Mężczyzna")\
           .when(df.gender.isNull(),"---").otherwise("NIE WIEM").alias("gender_descr")
          ).show()

+--------+---------+------------+
|   fname|    lname|gender_descr|
+--------+---------+------------+
|     Jan| Kowalski|   Mężczyzna|
|   Maria|    Nowak|     Kobieta|
|   Zenon|Martyniuk|         ---|
|Grzegorz|    Braun|   Mężczyzna|
+--------+---------+------------+



In [ ]:
data_rows = df.select("fname","lname").collect()
for row in data_rows:
  print(row["fname"], row["lname"])

Jan Kowalski
Maria Nowak
Zenon Martyniuk
Grzegorz Braun


In [ ]:
from pyspark.sql.functions import concat_ws, lit

df.select(
    concat_ws(",", df.fname, df.lname).alias("full_name") , df.gender, lit(True).alias("manager")
    ).show()

+---------------+------+-------+
|      full_name|gender|manager|
+---------------+------+-------+
|   Jan,Kowalski|     M|   true|
|    Maria,Nowak|     K|   true|
|Zenon,Martyniuk|  null|   true|
| Grzegorz,Braun|     M|   true|
+---------------+------+-------+



In [ ]:
def func(row):
     print(row.id)

df.foreach(func)

In [ ]:
df = spark.createDataFrame([(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])
def func(person):
  print(person.name)

df.foreach(func)

In [ ]:
# konwersja do Pandas DataFrame
dfp = df.toPandas()
for index, row in dfp.iterrows():
  print(index, row["name"], row["age"], sep="|")
  print("================")


0|Tom|14
1|Alice|23
2|Bob|16


In [ ]:
# Użycie Pandas w PySpark
import pyspark.pandas as pds
import pandas as pd

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
df_pandas = pd.read_csv("https://eurocash-pyspark.s3.eu-central-1.amazonaws.com/tips.csv")
df_pandas

In [ ]:
#pds.read_csv("https://eurocash-pyspark.s3.eu-central-1.amazonaws.com/tips.csv")

In [ ]:
pd.__version__

'1.5.3'

In [ ]:
!pip install pandas -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.


### Filtrowanie

In [ ]:
from pyspark.sql.types import ArrayType

data = [
    ( ("Jan","Kowalski") , ["Java","Python"], 44, "M" , "Warszawa" ),
    ( ("Maria","Nowak") ,  ["Java","C++"], 33, "K", "Gdańsk"   ),
     ( ("Zenon","Martyniuk") ,  ["Excel","Python"], 33, None,  "Poznań"   ),
     ( ("Grzegorz","Braun") ,  ["Pascal","Rust"], None, "K",  "Kraków"   ),
]

schema = StructType([
    StructField("name", StructType([
        StructField("firstname", StringType(), True),
        StructField("lastname", StringType(), True)
    ])),
    StructField("lang", ArrayType(StringType()), True ),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("city", StringType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()
df.show()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- lang: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)

+------------------+---------------+----+------+--------+
|              name|           lang| age|gender|    city|
+------------------+---------------+----+------+--------+
|   {Jan, Kowalski}| [Java, Python]|  44|     M|Warszawa|
|    {Maria, Nowak}|    [Java, C++]|  33|     K|  Gdańsk|
|{Zenon, Martyniuk}|[Excel, Python]|  33|  null|  Poznań|
| {Grzegorz, Braun}| [Pascal, Rust]|null|     K|  Kraków|
+------------------+---------------+----+------+--------+



In [ ]:
from pyspark.sql.functions import array_contains

df.filter( (df.age > 20) & (df.gender=="K")  ).show()

+--------------+-----------+---+------+------+
|          name|       lang|age|gender|  city|
+--------------+-----------+---+------+------+
|{Maria, Nowak}|[Java, C++]| 33|     K|Gdańsk|
+--------------+-----------+---+------+------+



In [ ]:
df.filter( array_contains(df.lang, "Python") ).show()

+------------------+---------------+---+------+--------+
|              name|           lang|age|gender|    city|
+------------------+---------------+---+------+--------+
|   {Jan, Kowalski}| [Java, Python]| 44|     M|Warszawa|
|{Zenon, Martyniuk}|[Excel, Python]| 33|  null|  Poznań|
+------------------+---------------+---+------+--------+



In [ ]:
df.filter( df.name.firstname == "Jan" ).show()

+---------------+--------------+---+------+--------+
|           name|          lang|age|gender|    city|
+---------------+--------------+---+------+--------+
|{Jan, Kowalski}|[Java, Python]| 44|     M|Warszawa|
+---------------+--------------+---+------+--------+



In [ ]:
df1 = df.select("name.firstname", "age")
df1.show()

+---------+----+
|firstname| age|
+---------+----+
|      Jan|  44|
|    Maria|  33|
|    Zenon|  33|
| Grzegorz|null|
+---------+----+



In [ ]:
df2 = df1.filter(df1.age.isNotNull())\
      .rdd\
      .map(lambda row: (row["firstname"], row["age"]*1.2) )\
      .toDF(["Name","Age"])
df2.show()

+-----+----+
| Name| Age|
+-----+----+
|  Jan|52.8|
|Maria|39.6|
|Zenon|39.6|
+-----+----+



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("Index", monotonically_increasing_id())
df.show()

+------------------+---------------+----+------+--------+----------+
|              name|           lang| age|gender|    city|     Index|
+------------------+---------------+----+------+--------+----------+
|   {Jan, Kowalski}| [Java, Python]|  44|     M|Warszawa|         0|
|    {Maria, Nowak}|    [Java, C++]|  33|     K|  Gdańsk|         1|
|{Zenon, Martyniuk}|[Excel, Python]|  33|  null|  Poznań|8589934592|
| {Grzegorz, Braun}| [Pascal, Rust]|null|     K|  Kraków|8589934593|
+------------------+---------------+----+------+--------+----------+



In [ ]:
df.filter( df.Index % 2 ==0 ).show()

+------------------+---------------+---+------+--------+----------+
|              name|           lang|age|gender|    city|     Index|
+------------------+---------------+---+------+--------+----------+
|   {Jan, Kowalski}| [Java, Python]| 44|     M|Warszawa|         0|
|{Zenon, Martyniuk}|[Excel, Python]| 33|  null|  Poznań|8589934592|
+------------------+---------------+---+------+--------+----------+



In [ ]:
df.orderBy( df.name.firstname.desc() , df.city.asc() ).show()

+------------------+---------------+----+------+--------+----------+
|              name|           lang| age|gender|    city|     Index|
+------------------+---------------+----+------+--------+----------+
|{Zenon, Martyniuk}|[Excel, Python]|  33|  null|  Poznań|8589934592|
|    {Maria, Nowak}|    [Java, C++]|  33|     K|  Gdańsk|         1|
|   {Jan, Kowalski}| [Java, Python]|  44|     M|Warszawa|         0|
| {Grzegorz, Braun}| [Pascal, Rust]|null|     K|  Kraków|8589934593|
+------------------+---------------+----+------+--------+----------+

